## Imports

In [1]:
import collections
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint
from nltk.tokenize import sent_tokenize
import networkx as nx
import gensim 
from gensim.models import Word2Vec 

## Funções

In [2]:
def word_tokenizer(text):
        tokens = word_tokenize(text)
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(t) for t in tokens if t not in stopwords.words('english')]
        return tokens


def get_closest_sentences_after_clustering(sentences, nb_of_clusters=5):
        tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenizer,
                                        stop_words=stopwords.words('english'),
                                        max_df=0.9,
                                        min_df=0.05,
                                        lowercase=True)
        tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
        kmeans = KMeans(n_clusters=nb_of_clusters)
        kmeans.fit(tfidf_matrix)
        closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, tfidf_matrix)
        return closest

In [3]:
def summarize_by_cluster_distance(sentences):
    nclusters= 3
    closest_indexes = get_closest_sentences_after_clustering(sentences, nclusters)
    buffer = ""
    for i in closest_indexes:
        buffer += str(sentences[i])
    return buffer

In [4]:
def summarize_with_text_rank(sentences):
    tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenizer,
                                        stop_words=stopwords.words('english'),
                                        max_df=0.9,
                                        min_df=0.05,
                                        lowercase=True)
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    matrix = tfidf_matrix.todense()
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            sim_mat[i][j] = cosine_similarity(matrix[i], matrix[j])[0,0]
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    n_of_sentences = 3
    top_sentences = []
    buffer = ""
    for i in range(n_of_sentences):
        sentence = ranked_sentences[i][1]
        top_sentences.append({"sentence": sentence, "index": sentences.index(sentence)})
    ordered_by_occurency = sorted(top_sentences, key=lambda k: k['index']) 
    for sentence_dict in ordered_by_occurency:
        buffer += sentence_dict['sentence']
    return buffer

In [5]:
def Get_Keywords(fullText):
        data = [] 
        words = []
        for i in sent_tokenize(fullText): 
            temp = [] 
            for j in word_tokenize(i): 
                temp.append(j.lower()) 
                words.append(j.lower())
            data.append(temp) 
        model2 = gensim.models.Word2Vec(data, min_count = 1, size = 100,sg = 1)    
        sim_mat = np.zeros([len(words), len(words)])
        for i in range(len(words)):
            for j in range(len(words)):
                sim_mat[i][j] = model2.similarity(words[i], words[j])
        try:
            nx_graph = nx.from_numpy_array(sim_mat)
            scores = nx.pagerank(nx_graph)
            ranked_words = sorted(((scores[i],s) for i,s in enumerate(words)), reverse=True)
            return ranked_words
        except:
            return "Não foi possível rankear as palavras, problemas de convergência"

## Geração do Dataset

In [6]:
import json
file1 = open('australian_user_reviews.json', 'r', encoding="utf8") 
lines = file1.readlines() 

documents = []
for line in lines: 
    jsonLine = line.strip()
    s = jsonLine.strip()
    l = eval(s)
    for i in l['reviews']:
        documents.append(i['review'])

In [7]:
len(documents)

59305

In [8]:
filtred_documents = []
sentences_threshold = 15
for x in documents:
    if(len(sent_tokenize(x)) >= sentences_threshold):
        filtred_documents.append(x)
print("Documentos com sentenças minimas: ", len(filtred_documents))

Documentos com sentenças minimas:  766


## Sumarização

In [9]:
counter = 1
samples_to_test = 20
for document in filtred_documents[:samples_to_test]:
    sents = sent_tokenize(document)
    print("Document: {0}\n".format(counter))
    print("Texto: \n")
    print(document + "\n")
    try:
        resumo = summarize_by_cluster_distance(sents)
        print("Resumo  com algoritmo distancia centroids:\n")
        print(resumo)
        print("\n")
        resumo = summarize_with_text_rank(sents)
        print("Resumo  com text rank:\n")
        print(resumo)
        print("\n\n")
    except:
        print("Erro: ", sents)
    counter += 1

Document: 1

Texto: 

Buy it. Support it. It's awesome. I will find you if you don't.^ Kek, what an old report.Personally I think you shouldn't play this game, although it might get better in a few years.I bought this game as soon as it came out, I bought the founders two pack. It was fun, pretty buggy, but it was full of promise. Skip forward a few years later. The game is worse. Lets talk about starting the game. The game is slow to start. Starting a game will take just as long, and when you get into the game itself, prepare for unoptimized slowness. The game runs terribly, is riddled with bugs and the devs generally don't add any new content. If this was developed by a good indie team, or an AAA team (Oh, did i forgot to mention they fired several people who worked at Bioware, and hired some scrubs?), it would be amazing. The concept is great, I would have really looked forward to this game shining, but the devs are horrible. I do not recommend this game.



C:\Users\hugos\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Resumo  com algoritmo distancia centroids:

The concept is great, I would have really looked forward to this game shining, but the devs are horrible.The game is slow to start.It's awesome.




C:\Users\hugos\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Resumo  com text rank:

The game is slow to start.Starting a game will take just as long, and when you get into the game itself, prepare for unoptimized slowness.The concept is great, I would have really looked forward to this game shining, but the devs are horrible.



Document: 2

Texto: 

I'd definitely recommend this game for people who want to expand their vocabulary, but there are a few things I'd like to point out.-While this is almost never required for romance languages, in Japanese, Chinese and Korean, words, Especially adjectives and verbs, can be synonymous but imply different things. This is almost always the case in Japanese, where verbs have transitive and intransitive forms, but this is not listed in the definition, only one form is presented. This means doing research on the word you'll find yourself using more will help you be more fluent in the long run.-So much katakana... I'd like to see really rare kanji forms of words, as they can appear in stories and some video

...

Resumo  com text rank:

Strongmen with a fearof the dark!Never-spoiling Twinkies!Or terraria!



Document: 20

Texto: 

STOP! Before you go anywhere, read this. This game is worth playing. Why? Well, imagine this. Your name is Lee Everett. A thirty-something year old man on his way to prison for an undisclosed crime. The game begins with you getting driven out of the city of Atlanta by an elderly policeman. This is a tutorial of sorts as you learn how to look around and click on objects of interest. You also learn about the dialogue system in which you get given 4 options of words/sentences to reply with. Choose as you will, the copper looks away at a bad moment and crashes into a pedestrian, spins out of control and tumbles down a steep ditch on the side of the road. Dazed, confused and hurt you pass out for a few minutes to the sound of screaming. This is where the game really begins. I won't spoil much but this is a really enjoyable game. 9.5/10 is a score that this game earns excep

C:\Users\hugos\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Resumo  com algoritmo distancia centroids:

Well, imagine this.You also learn about the dialogue system in which you get given 4 options of words/sentences to reply with.This is where the game really begins.




C:\Users\hugos\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Resumo  com text rank:

This game is worth playing.Well, imagine this.This is where the game really begins.





## Extração de palavras chaves

In [10]:
counter = 1
samples_to_test = 20
for document in filtred_documents[4:6]:
    print("Document: {0}\n".format(counter))
    print("Texto: \n")
    print(document + "\n")
    try:
        keywords = Get_Keywords(document)
        print("Palavras chaves de um texto:\n")
        print(keywords)
        print("\n\n")
    except:
        print("Erro: ", sents)
    counter += 1

Document: 1

Texto: 

Is your computer BAD? Do you want the best games but your computer can't handle it? Well this games for you! Its basically DayZ and Minecraft squished together! So if your computer can't hold DayZ, Come on down and try this! Loot towns! Build bases! Fight the Zombies that are everywhere! Yes everywhere even above you... Wait no that was a lie. * Continues *. This is endless fun! Play with your friends if your to scared to fight the little zombies. Except for the one behind you! Wait that was a lie again.. Well then, what are you waiting for? Download this game right now and then play it then get Gold Upgrade then have endless fun and oh by the way theres a zombie behind you to your left.. Wait that was a lie. Or was it...Lets end this. This is a brilliant game and you should get it if you love DayZ but can't handle it on your CP. Goodbye!



C:\Users\hugos\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


Palavras chaves de um texto:

Não foi possível rankear as palavras, problemas de convergência



Document: 2

Texto: 

To commemorate tha beta of Steam Reviews, it only make sense dat I finally write a recommendation fo' a game dat has been close ta mah ♥♥♥ since tha dizzle I was first introduced ta it: Terraria.Terraria has proven ta be a ♥♥♥♥♥♥♥♥ of funk ta play, n' has been one of mah freshest vizzle game time sinks up in recent mutha♥♥♥♥in years cuz of dat ♥♥♥♥. Between lenient system requirements, a solid (albeit imbalanced, at times) combat system, a funky-♥♥♥ broad permittizzle fo' exploration n' a wide selection of buildin decor, itz tha type of game dat strikes me as havin suttin' fo' everyone. Which is good, when you consider dat it also has multiplayer n' ♥♥♥♥. Even mo' betta when you git ta tha late game, which has a wide array of weapons, armors n' tools ta allow fo' a wide variety of specializations.Buildin n' exploration is phat points up in favor of tha game yo, but tha